## 4. Time evolution
We now turn our attention to larger systems and to time evolution. Remember last week, when we diagonalized the Hamiltonian of a particle in a box? Today we will see where that comes from, as well as a bit on the Schrödinger equation.

### 4.1 How to write a Hamiltonian
Writing a Hamiltonian is an important thing to know. Once we have the Hamiltonian, we can compute the time evolution operator and with that we can fast-forward time and rewind the clock on any quantum state!

We need to begin from the role of the Hamiltonian as the Observable of energy. Energy in mechanical systems (like a particle in a box) comes in two forms: kinetic and potential: $H = H_\mathrm{kinetic} - H_\mathrm{potential}$. Classically, the kinetic energy can be written as

$$
E_\mathrm{kinetic} = \frac{P^2}{2m}
$$

Where $P$ is the momentum and $m$ is the mass of the particle (which is just a numerical parameter). So in order to write the kinetic contribution to the Hamiltonian, we need the momentum Observable. What is the momentum observable? To answer this question we need to go back for a second to the fact that position and momentum are continuous quantities, and their Observable are actually defined in an infinite-dimensional Hilbert space. This is very convenient because in an infinite-dimensional Hilbert space we can do calculus (integrals, derivatives and so on). In fact, those are our only tools to define operators (which in that case cannot be matrices, because the space is infinite-dimensional).

So what we'll do is we will find the momentum Observable $P$ with a trick. First of all, we'll need something that contains $P$. Luckily we know that the unitary operator that shifts the position of a particle is generated by the momentum Observable:

$$
U(x) = \exp(i x P)
$$

So we can write:

$$
-i\frac{\partial}{\partial x}U(x) = P\exp(ixP) = PU(x)
$$

Which means that:

$$
-i\frac{\partial}{\partial x} = P
$$

and therefore 
$$
P^2 = -\frac{\partial^2}{\partial x^2}
$$

This makes perfect sense in an infinite-dimensional vector space, because the operators on it are in differential form (they are not matrices). But what happens when we discretize the space? We saw that integrals become sums, what about derivatives? This is simpler than one may think: a derivative is the limit for $\delta x\rightarrow 0$ of $$\frac{\psi(x + \delta x) - \psi(x)}{\delta x}$$ In a discrete space though, we can't have $\delta x$ go all the way to zero! It will have to shrink at most to the size of the smallest interval. But then $\psi(x+\delta x)$ and $\psi(x)$ are two amplitudes that are next to each other in the discrete space. So that tells us how to construct the matrix the implements the discrete derivative: 

$$
\frac{\partial}{\partial x} = 
\begin{pmatrix}
-1 & 1 & 0 & 0 & \cdots\\
0 & -1 & 1 & 0 &\cdots\\
0 & 0 & -1 & 1 & \cdots\\
\vdots & \vdots & \vdots& \ddots & \ddots
\end{pmatrix}
$$

This is a well-known technique in finite element analysis. Okay, we are almost done! To obtain the second derivative we can apply the formula for the (symmetric) second derivative $\frac{1}{\delta x^2}(\psi(x-\delta x) - 2\psi(x) + \psi(x+\delta x))$ and we obtain the matrix that implements it:

$$
\frac{\partial^2}{\partial x^2} = 
\begin{pmatrix}
-2 & 1 & 0 & 0 & \cdots\\
1 & -2 & 1 & 0 &\cdots\\
0 & 1 & -2 & 1 & \cdots\\
0 & 0 & 1 & -2 & \ddots\\
\vdots & \vdots & \vdots& \ddots & \ddots
\end{pmatrix}
$$

With this, we can finally build the symmetric Hamiltonian:

$$
H = \frac{1}{2m}
\begin{pmatrix}
2 & -1 & 0 & 0 & \cdots\\
-1 & 2 & -1 & 0 &\cdots\\
0 & -1 & 2 & -1 & \cdots\\
0 & 0 & -1 & 2 & \ddots\\
\vdots & \vdots & \ddots& \ddots & \ddots
\end{pmatrix}
$$

And that tells us how to construct the kinetic term of the Hamiltonian! The potential term is easy (especially in our case). If we have a potential that depends on the position of the particle (like a potential well, or a harmonic oscillator like a mass on a spring etc...) then it will be diagonal in the position basis. It will simply be a diagonal matrix with the value of the potential $V(x)$ at the discrete position $x$. A box with solid boundaries has zero potential inside and infinite potential outside. So for us it's trivial to implement the walls of the box: they correspond to the first and last index! Our quantum state is guaranteed to stay in the box because there are no indices to describe positions outside of it! So here's our Hamiltonian:

Finally, because diagonalization algorithms can get picky when things are not symmetric, we will shift

---
#### Activity 4: Implement the Hamiltonian (10 minutes)
Write a function that takes the mass of the particle and the dimension of the Hilbert space and returns the matrix of the Hamiltonian. The signature should be `f(float, int) -> np.array(complex)`. TIP: the function `np.diag()` has a second argument that you can use to fill the diagonal of your choice, not just the central one.

---

In [ ]:
def H(m, dim):
    P_squared = np.diag(2*np.ones(dim), k=0) + np.diag(-1*np.ones(dim-1), k=1) + np.diag(-1*np.ones(dim-1), k=-1)
    return P_squared/(2*m)

### 4.2 How to compute the action of the time evolution operator
We are going to see two ways of computing time evolution.

The first way to compute the action of $U(t)$ on a state is to exponentiate the Hamiltonian for a fixed choice of time interval $\Delta t$, and the resulting matrix advances time by $\Delta t$ every time we use it to multiply the state:

$$
\begin{align}
U(\Delta t)|\psi(0)\rangle &= |\psi(\Delta t)\rangle\\
U(\Delta t)|\psi(\Delta t)\rangle &= |\psi(2\Delta t)\rangle\\
\mathrm{etc}\dots
\end{align}
$$

So we need to compute $U(\Delta t)$ only once and for all.

If we want more flexibility, or for example to have $|\psi(t)\rangle$ for all times $t$ without having to recompute a new matrix exponential for all $t$, we can adopt the second way. We express $|\psi(0)\rangle$ in the eigenbasis of $H$ and then we multiply the $k$-th amplitude by $\exp(it\lambda_k)$. This is a much simpler calculation than a whole matrix exponential (so it's much faster) and it gives us $|\psi(t)\rangle$ directly.

Let's implement both. We start by computing $U(\Delta t)$ for a small time interval, say $\Delta t=1$ (this is a small interval in relation to the energy):

In [ ]:
U = expm(1j*1.0*H(0.5, 200))

In [ ]:
psi = np.array([np.exp(-(x-100)**2/(2*25)) for x in range(200)])
psi = psi/np.linalg.norm(psi)

In [ ]:
speed = 100
state = psi
for k in range(100):
    plt.plot(abs(state)**2)
    state = U@state

I've written a function (very inefficient as it creates a new figure object each time) to create a "live plot". If you find a way to make it faster (for instance by reusing the same figure object) let me know!

In [ ]:
from collections import defaultdict
data = defaultdict(list)

psi = np.array([np.exp(-(x-100)**2/(2*25)) for x in range(200)])
speed = 100
state = psi*np.exp(-1j*speed*np.linspace(-1,1,200))

for i in range(100):
    state = U@state
    data['prob']= np.abs(state)**2
    live_plot(data)

Now we look at the second method:

In [ ]:
eigenvalues, Vdagger = np.linalg.eig(H(0.5, 200))

psi = np.array([np.exp(-(x-100)**2/(2*25)) for x in range(200)])

# the wave function in the eigenbasis of H
psi_H = np.conj(Vdagger.T)@psi

In [ ]:
def psi_evolved(t):
    return Vdagger@(np.exp(1j*eigenvalues*t)*psi_H)

In [ ]:
data = defaultdict(list)

for t in range(100):
    data['prob']= np.abs(psi_evolved(t))**2
    live_plot(data)